# Community Detection Process
* Pipeline for determining gamma range to run community detection on
* Factors that contribute to determining gamma range:
    - (1) Percentage of nodes in the top 8 modules
    - (2) How well a partition recovers the reference partition
    - (3) How much of the Auditory network gets separated into it's own module

In [ ]:
from collections import Counter
import pandas as pd
import importlib
import sys
import matplotlib.pyplot as plt
import data_io as myFunc
import functional_brain_community as brain_community
import pickle as pkl
import os
import OVARIAN_functions as ov_fct
importlib.reload(ov_fct)
importlib.reload(myFunc)
importlib.reload(brain_community)
print(sys.version)

# Update your own folder paths.
PICKLE_PATH = '../Ovarian_hormone/pickles/'
FIGURE_PATH = '../Ovarian_hormone/Figures/'

def split_ranges(grange):
    """
    Split a given range (or a list of ranges) into segments of 50 iterations.
    Only need this function for running community detection on ARC. With 50 segment blocks,
    each ARC script should complete running within 18 hours.
    :param grange: a list of values, with each pair considered as a range.
    :return: list of tuples of separated ranges.
    """
    j = 0
    ranges = []
    while j < len(grange):
        lower = grange[j]
        upper = grange[j+1]
        n = round((upper-lower)/0.001)
        j += 2
        if n == 50:
            ranges.append((lower, upper+0.001))
        else:
            while n > 50:
                ranges.append((lower, lower + 50*0.001))
                lower = lower + 50*0.001
                if lower == upper:
                    ranges.append((lower, upper+0.001))
                    n = 0
                    break
                else:
                    n = round((upper-lower)/0.001)
            if n > 0:
                ranges.append((lower, upper+0.001))
    return ranges

def community_detection2(connectome, gamma):
    """
    Community detection using Louvain modularity with 20 repetitions.
    :param connectome: graph for community detection
    :param gamma: value of gamma for Louvain modularity
    :return: 20 partitions, their modularity value, and stability value
    """
    partitions, modularity, stability = brain_community.community_by_modularity_stability(connectome, gamma, B='negative_asym', rep = 20)
    return partitions, modularity, stability

def find_icns(reference_partition, reference_FN_assignment, partition):
    """
    Given a community breakdown of a network, automate assignment of communities to
    functional networks (ICNs) in the brain based on a reference network.
    :param reference_partition: Desired/reference partition of a network
    :param reference_FN_assignment: Corresponding functional network assignment for the reference partition (dictionary labelling)
    :param partition: partition of modules to be assigned an ICN based on the reference partition structure
    :return: reindexed partition with modules that matches the reference ICN assignment,
    percent of overlap compared with the reference partition,
    average overlap across all ICNs
    """
    ref_modules = ov_fct.get_modules(reference_partition, reference_FN_assignment)
    my_modules, partition_norm = ov_fct.get_all_modules(partition)
    ref_preference = ov_fct.get_preference_list(ref_modules, my_modules)
    pref = ov_fct.get_preference_list(my_modules, ref_modules)
    matching = ov_fct.get_poly_matching(pref, ref_preference, poly_threshold=0.6)
    merged_partition = ov_fct.merge_poly_modules(partition_norm, matching)
    new_modules = ov_fct.get_modules(merged_partition, reference_FN_assignment)

    inverse_map = {v: k for k, v in reference_FN_assignment.items()}
    fn_overlap = {}
    avg_overlap = 0
    for fn_name in ['Visual', 'Somamotor', 'DMN', 'Auditory']:
        fn_idx = inverse_map[fn_name]
        overlap = ref_modules[inverse_map[fn_name]].intersection(new_modules[fn_idx])
        denom = len(ref_modules[inverse_map[fn_name]])
        overlap = (len(overlap)*1.0)/(denom*1.0)
        fn_overlap[fn_name] = overlap
        if fn_name in ['Visual', 'Auditory']:
            avg_overlap += overlap
    return merged_partition, fn_overlap, avg_overlap

def find_optimal_range(reference_partition, reference_FN_assignment, connectome):
    min_g = 1.1
    max_g = 1.3
    i = 0
    history = []
    while (max_g-min_g)/0.001 > 100:
        partitions, modularity, stability = community_detection2(connectome, min_g)
        min_partition = brain_community.get_best_partition(partitions)
        min_merged_partition, min_fn_overlap, min_avg_overlap = find_icns(reference_partition, reference_FN_assignment, min_partition)

        # data = {'partition': min_merged_partition, 'fn_overlap': min_fn_overlap, 'g': min_g}
        # myFunc.save_to_pickle(data, PICKLE_PATH + 'EF-039-190926/', f'EF-039-190926_min_{i}')

        partitions, modularity, stability = community_detection2(connectome, max_g)
        max_partition = brain_community.get_best_partition(partitions)
        max_merged_partition, max_fn_overlap, max_avg_overlap = find_icns(reference_partition, reference_FN_assignment, max_partition)

        top_8 = [y for x,y in Counter(min_partition).most_common(8)]
        top_8_sum = sum(top_8)
        print(f"\t Min gamma {min_g}: {len(Counter(min_partition))} communities. {top_8_sum*100/1054:0.2f}% are in top 8.", min_fn_overlap)
        if (top_8_sum*100/1054) > 70:
            history.append((min_g, top_8_sum*100/1054, min_fn_overlap['Auditory']))

        top_8 = [y for x,y in Counter(max_partition).most_common(8)]
        top_8_sum = sum(top_8)
        print(f"\t Max gamma {max_g}: {len(Counter(max_partition))} communities. {top_8_sum*100/1054:0.2f}% are in top 8.", max_fn_overlap)
        top_8_max = top_8_sum*100/1054
        if (top_8_sum*100/1054) > 70:
            history.append((max_g, top_8_sum*100/1054, max_fn_overlap['Auditory']))

        i += 1
        if min_fn_overlap['Visual']>=0.75 and min_fn_overlap['Auditory'] >= 0.5:
            max_g = min_g + 0.12
            break
        elif max_fn_overlap['Visual']>=0.75 and max_fn_overlap['Auditory'] >= 0.5:
            min_g += 0.05
        else:
            if top_8_max < 70:
                # Go through the history list and find the best gamma we have
                best_history = sorted(history, key = lambda x: x[2], reverse=True)[0]
                print("Best history: ", best_history)
                return best_history[0] - 0.06, best_history[0] + 0.06
            else:
                min_g = max_g
                max_g += 0.12
                print(f"Skipping new range to: {min_g} - {max_g}")

    if (max_g - min_g)/0.001 < 120:
        return max_g - 0.12, max_g
    return min_g, max_g

def load_connectome(idx, phase):
    path = f'../Ovarian_hormone/pickles/individual_connectomes/{idx}/'
    for root, dirs, files in os.walk(path):
        for file in files:
            if not file.endswith('.pkl'):
                continue
            if f'{phase}' in file:
                with open(root+file, 'rb') as f:
                    return pkl.load(f)
    return []

ef_input_params = myFunc.load_from_pickle(PICKLE_PATH+"/individual_connectomes/", "ef_input_params.pkl")
ef_name_to_idx = {x[0]: x[1] for x in ef_input_params.values()}

# Estimating Gamma
- Iterate through a large range of gamma values to see which gamma ranges best produce a community structure with our biological requirements.
- Split the best gamma range into blocks of 50 iterations and pickle it.
This pickle will be the input to the ARC community detection script.

In [ ]:
all_cycles = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'cycles_list.pkl')
bad_ef = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'bad_subject_ef.pkl')
bad_ef_list = [x[0] for x in bad_ef]
reference_FN_assignment = ov_fct.AVG_EF_FN_AUDITORY
# reference_partition = myFunc.load_from_pickle(PICKLE_PATH, f'EF_best_partition_auditory.pkl')
input_params = {}
fname_i = 1

# input_params = myFunc.load_from_pickle('../Ovarian_hormone/pickles/individual_connectomes/', 'ef_input_params.pkl')
for ef_name, lf_name, ml_name in all_cycles:
    if ef_name+'.pkl' in bad_ef_list:
        continue
    if fname_i not in [1,2,3,4,5,7,9,10,13,15,18]:
        fname_i += 1
        continue
    print(fname_i, ef_name, lf_name, ml_name)
    idx = ef_name_to_idx[ef_name+".pkl"]

    # ********************************************************************
    reference_partition = myFunc.load_from_pickle(PICKLE_PATH+"/best_subject_auditory/", f'{ef_name}_auditory.pkl')
    connectome = load_connectome(idx, lf_name)
    print(f"Processing {lf_name}")
    # ********************************************************************

    min_r, max_r = find_optimal_range(reference_partition, reference_FN_assignment, connectome)
    gamma_ranges = split_ranges([min_r, max_r])
    print("Returned ranges: ", min_r, max_r, gamma_ranges)
    if len(gamma_ranges) != 3:
        print("****** WARNING ******")
        print("\n"*2)
    input_params[fname_i] = [ef_name+".pkl", idx, gamma_ranges]
    fname_i += 1
    # Save
    myFunc.save_to_pickle(input_params, '../Ovarian_hormone/pickles/individual_connectomes/', 'lf_input_params_temp')
